# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。此时，我们的目标是找到使损失函数最小化的模型参数值。 经过训练后，我们将需要使用这些参数来做出未来的预测。此外，有时我们希望提取参数，以便在其他环境中复用它们，将模型保存下来，以便它可以在其他软件中执行，或者为了获得科学的理解而进行检查。   

 
本节，我们将介绍以下内容：
- 访问参数，用于调试、诊断和可视化
- 参数初始化
- 在不同模型组件间共享参数


首先来看具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3026],
        [-0.2636]], grad_fn=<AddmmBackward0>)

### 5.3.1参数访问
我们从已有模型中访问参数。当通过`Sequential`类定义模型时，我们可以通过索引来访问模型的任意层。这就像模型时一个列表一样，每层的属性都在其属性中。如下所示，我们可以检查第二个全连接层的参数。

In [5]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1224,  0.2770, -0.2231, -0.2671, -0.2894,  0.2660,  0.1554, -0.0884]])), ('bias', tensor([-0.2627]))])


输出的结果告诉我们一些重要的事情：首先，这个全连接层包含两个参数，分别是该层的权重和偏置。两者都存储为单精度浮点数。注意，参数名称允许唯一标识每个参数，即使在包含百个层的网络中也是如此。

#### 5.3.1.1 目标参数
注意每个参数都表示为参数类的一个实例。要对参数执行任何操作，首先我们需要访问底层的数值。有几种方法可以做到这一点。     
下面的代码从第二个全连接层(即第三个神经网络)提取偏置，提取后返回的是一个参数实例，并进一步访问该参数的值。

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2627], requires_grad=True)
tensor([-0.2627])


参数时复合的对象，包含值、梯度和额外信息。这就是我们显式参数值的原因。除了值之外，我们还可以访问每个参数的梯度。在上面这个网络中，由于我们还没有调用反向传播，所以参数梯度处于初始状态。

In [7]:
net[2].weight.grad == None

True

#### 5.3.1.2 一次性访问所有参数
当我们需要对所有参数执行操作时，逐个访问可能比较麻烦。当我们处理比较复杂的块(例如，嵌套块)时，情况可能会变得比较复杂，因为我们可能需要递归整个树来提取每隔子块的参数。下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。

In [9]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这位我们提供了另一种访问网络参数的方式，如下所示。

In [40]:
net.state_dict()['2.bias'].data

tensor([0.])

#### 5.3.1.3 从嵌套块收集参数

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2827],
        [-0.2827]], grad_fn=<AddmmBackward0>)

我们观察它是如何工作的

In [14]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [15]:
rgnet[0][1][0].bias.data

tensor([ 0.4476, -0.2005, -0.4433,  0.3934,  0.2849, -0.2800, -0.3963,  0.4434])

In [19]:
rgnet.state_dict()['0.block2.2.weight']
# rgnet.state_dict()

tensor([[-0.1160, -0.0012, -0.0685,  0.0453, -0.0415, -0.0515, -0.2310, -0.1009],
        [ 0.1928, -0.2719,  0.2937, -0.0444,  0.0247,  0.1526, -0.0329,  0.1980],
        [ 0.0575, -0.2455, -0.2894,  0.2438, -0.1333, -0.2149,  0.1101,  0.2353],
        [ 0.1517,  0.1477,  0.1908,  0.2851, -0.1151, -0.2417,  0.0547,  0.1338]])

### 5.3.2 参数初始化
知道了如何访问参数后，现在我们看看如何正确的初始化参数。深度学习框架允许提供默认随机初始化，也允许我们创建自定义初始化方法，满足我们通过其他规则实现初始化权重。

#### 5.3.2.1 内置初始化
调用内置的初始化器。下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，且将偏置参数设置为0。

In [24]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0095,  0.0029, -0.0027,  0.0015]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1.

In [26]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以对某些块应用不同的初始化方法。例如下面我们使用`Xavier`初始化方法初始化第一个神经网络层，然后将第三个神经网络层初始化为常量42.

In [27]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data[0])


tensor([-0.2303,  0.0427,  0.3993,  0.4682])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


#### 5.3.2.2 自定义初始化
有时深度学习框架并没有为我们提供需要的初始化方法。在下面的例子中，我们使用以下的分布为任意权重参数`w`定义初始化方法：

$$
w~~ \begin{cases}
        U(5, 10) & 可能性 \frac{1}{4} \\
        0 & 可能性 \frac{1}{2}        \\
        U(-10, -5) & 可能性 \frac{1}{4}
\end{cases}
$$

In [37]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2], net[2].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


(tensor([[ 7.9477,  0.0000,  0.0000, -7.0191],
         [-0.0000, -5.7341,  0.0000, -7.5426]], grad_fn=<SliceBackward0>),
 Parameter containing:
 tensor([[6.8970, 0.0000, 9.4114, 0.0000, -0.0000, -0.0000, 7.1320, 0.0000]],
        requires_grad=True))

注意，我们始终可以直接设置参数。

In [41]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -6.0191])